In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

import TheoreticallyOptimalStrategy as TOS
from marketsimcode import compute_portvals, compute_stats
import indicators
from strategy_evaluation.ManualStrategy import ManualStrategy
from strategy_evaluation.StrategyLearner import StrategyLearner

In [2]:
symbol = "JPM"
in_sample_sd = dt.datetime(2008, 1, 1)
in_sample_ed = dt.datetime(2009, 12, 31)
out_sample_sd = dt.datetime(2010, 1, 1)
out_sample_ed = dt.datetime(2011, 12, 31)
sv = 100000
commission = 9.95
impact = 0.05

#MANUAL STRATEGY

In [92]:
# Manual Strategy

sd = out_sample_sd
ed = out_sample_ed
indicator = indicators.Indicators(symbol, pd.date_range(sd, ed))
bbp = indicator.get_bbp(window = 20)
rsi = indicator.get_rsi(window = 14)
macd = indicator.get_macd().loc[sd:] # My version of indicators.py is also returning values 9 market days prior to sd

# Calculate MACD crossover
macd_prev = macd.shift(1)
macd_prev.iloc[0] = macd_prev.iloc[1]
conditions = [
    (macd_prev <= 0) & (macd > 0),
    (macd_prev >= 0) & (macd < 0)
]
values = [1, -1]
# Documentation: https://numpy.org/doc/2.1/reference/generated/numpy.select.html
macd_crossover = pd.DataFrame({'result': np.select(conditions, values, default=0)}, index=macd.index, columns=["result"])

# Generate signals
signals_cond = [
    ((bbp <= 0) & (rsi <= 30)) | ((bbp <= 0) & (macd_crossover["result"] == 1)) | ((rsi <= 30) & (macd_crossover["result"] == 1)),
    ((bbp >= 100) & (rsi >= 70)) | ((bbp >= 100) & (macd_crossover["result"] == 1)) | ((rsi >= 70) & (macd_crossover["result"] == 1))
]
signal_values = [1, -1]
signals = pd.DataFrame({'result': np.select(signals_cond, signal_values, default=0)}, index=bbp.index)

# Based on indicators on T day, we will place orders on T+1 day
signals = signals.shift(1)
signals = signals.fillna(0)
signals.drop(signals.index[-1])

# Generate Manual Strategy trades
net_position = 0
manual_trades = pd.DataFrame(0, index=bbp.index, columns=[symbol])

for date in signals.index:
    if signals.loc[date].iloc[0] == 1:
        manual_trades.loc[date] = 1000 - net_position
    elif signals.loc[date].iloc[0] == -1:
        manual_trades.loc[date] = -1000 - net_position

    net_position += manual_trades.loc[pd.to_datetime(date), symbol]

# portvals = compute_portvals(manual_trades, start_val=sv, commission=commission, impact=impact)
portvals = compute_portvals(manual_trades, start_val=sv, commission=0, impact=0)

print(compute_stats(portvals))

# manual_trades
portvals

(0.1531, 0.00031114650156924295, 0.007558240295161501, 0.6534983533034617)


2010-01-04    100000.0
2010-01-05    100000.0
2010-01-06    100000.0
2010-01-07    100000.0
2010-01-08    100100.0
                ...   
2011-12-23    115620.0
2011-12-27    115090.0
2011-12-28    114720.0
2011-12-29    115470.0
2011-12-30    115310.0
Length: 504, dtype: float64

In [36]:
compute_stats(portvals)

(-0.570665, -0.0016337761780937103, 0.05524187128897186, -0.46948794792624565)

#STRATEGY LEARNER

In [4]:
symbol = "JPM"
in_sample_sd = dt.datetime(2008, 1, 1)
in_sample_ed = dt.datetime(2009, 12, 31)
out_sample_sd = dt.datetime(2010, 1, 1)
out_sample_ed = dt.datetime(2011, 12, 31)
sv = 100000
commission = 9.95
impact = 0.005

manualstrategy = ManualStrategy()
strategylearner = StrategyLearner(impact=0.005, commission=9.95)
# trades = manualstrategy.testpolicy(symbol="JPM", sd=in_sample_sd, ed=in_sample_ed)
strategylearner.add_evidence(symbol="JPM", sd=in_sample_sd, ed=in_sample_ed)
trades = strategylearner.testPolicy(symbol="JPM", sd=in_sample_sd, ed=in_sample_ed)

portvals = compute_portvals(trades, start_val=sv, commission=commission, impact=impact)
# portvals = compute_portvals(trades, start_val=sv, commission=commission, impact=impact)

print(compute_stats(portvals))
portvals


EPOCH:  0

Reward:  1191.7647685003321

Scores & Count:  [1191.7647685003321] 0



EPOCH:  1

Reward:  1487.8003001556367

Scores & Count:  [1191.7647685003321, 1487.8003001556367] 1



EPOCH:  2

Reward:  1659.8906788808663

Scores & Count:  [1191.7647685003321, 1487.8003001556367, 1659.8906788808663] 2



EPOCH:  3

Reward:  1659.8906788808663

Scores & Count:  [1191.7647685003321, 1487.8003001556367, 1659.8906788808663, 1659.8906788808663] 3



EPOCH:  4

Reward:  1659.8906788808663

Scores & Count:  [1191.7647685003321, 1487.8003001556367, 1659.8906788808663, 1659.8906788808663, 1659.8906788808663] 4


            JPM
2008-01-02    0
2008-01-03    0
2008-01-04    1
2008-01-07    2
2008-01-08    1
...         ...
2009-12-24    1
2009-12-28    1
2009-12-29    1
2009-12-30    1
2009-12-31    1

[505 rows x 1 columns]
(-32.3464785, 0.005209648747123686, 0.12089472584628815, 0.684071281343413)


2008-01-02     100000.00
2008-01-03     100000.00
2008-01-04      89863.35
2008-01-07      69966.25
2008-01-08      51204.15
                 ...    
2009-12-24   -3134437.85
2009-12-28   -3134597.85
2009-12-29   -3134817.85
2009-12-30   -3134777.85
2009-12-31   -3134647.85
Length: 505, dtype: float64